# Pyspark_learning_notes

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
sc.master

'local[*]'

## textFile读取本地文件和hdfs文件

In [3]:
textfile=sc.textFile("file:/c:/f/test.txt")#本地文件注意路径书写格式file:

In [4]:
textfile.collect()

['300', '200', '22']

In [5]:
textfile.count()

3

In [8]:
textfile=sc.textFile("hdfs:/hdfsfile/test/test.txt")#hdfs文件注意路径书写格式hdfs:

In [9]:
textfile.collect()

['300', '200', '22']

## RDD的转换及运算

In [10]:
intRDD=sc.parallelize(list((3,1,2,5,5)))#创建RDD

In [11]:
intRDD

ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:175

In [12]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [13]:
strRDD=sc.parallelize(list(('app','note','oran','app','grap')))

In [14]:
strRDD.collect()

['app', 'note', 'oran', 'app', 'grap']

### map的用法

In [15]:
intRDD.map(lambda x:x+1).collect()

[4, 2, 3, 6, 6]

In [16]:
strRDD.map(lambda x:'ncol_'+x).collect()

['ncol_app', 'ncol_note', 'ncol_oran', 'ncol_app', 'ncol_grap']

### filter筛选

In [17]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [18]:
intRDD.filter(lambda x:x>=3).collect()

[3, 5, 5]

In [19]:
intRDD.filter(lambda x:1<x<=3).collect()

[3, 2]

In [20]:
strRDD.filter(lambda x: 'ap' in x).collect()

['app', 'app', 'grap']

### distinct删除重复项

In [21]:
intRDD.distinct().collect()

[1, 5, 2, 3]

In [22]:
strRDD.distinct().collect()

['app', 'note', 'oran', 'grap']

### randomSplit将整个集合的元素以随机数的方式按照比例分为多个RDD

In [23]:
strRDD.collect()

['app', 'note', 'oran', 'app', 'grap']

In [24]:
sRDD=strRDD.randomSplit([0.4,0.6])

In [25]:
sRDD[0].collect()

['app']

In [26]:
sRDD[1].collect()

['app', 'note', 'oran', 'grap']

### groupBy聚合运算

In [27]:
gRDD=intRDD.groupBy(lambda x:'even' if (x%2==0) else 'odd').collect()#按奇偶分离

In [28]:
gRDD

[('even', <pyspark.resultiterable.ResultIterable at 0x1a5bf22c898>),
 ('odd', <pyspark.resultiterable.ResultIterable at 0x1a5bf22c048>)]

In [29]:
gRDD[0][0],sorted(gRDD[0][1])#这里的sorted方法也可用list、tuple方法

('even', [2])

In [30]:
gRDD[1][0],sorted(gRDD[1][1])

('odd', [1, 3, 5, 5])

### union合并运算

In [31]:
intRDD1=sc.parallelize([3,1,2,5,5])
intRDD2=sc.parallelize([5,6])
intRDD3=sc.parallelize([2,7])

In [32]:
intRDD1.union(intRDD2).collect()

[3, 1, 2, 5, 5, 5, 6]

In [33]:
intRDD1.union(intRDD2).union(intRDD3).collect()

[3, 1, 2, 5, 5, 5, 6, 2, 7]

### intersection交集运算

In [34]:
intRDD1.intersection(intRDD2).collect()

[5]

### subtract差集运算

In [35]:
intRDD1.subtract(intRDD2).collect()

[1, 2, 3]

### cartesion笛卡尔合并

In [36]:
intRDD2.cartesian(intRDD3).collect()

[(5, 2), (5, 7), (6, 2), (6, 7)]

## RDD基本“动作”运算

In [37]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [38]:
intRDD.first()

3

In [39]:
intRDD.take(2)#取前两项

[3, 1]

In [40]:
intRDD.takeOrdered(3)#取前三项并排序

[1, 2, 3]

In [41]:
intRDD.takeOrdered(3,key=lambda x:-x)#从大到小排序后取前三项

[5, 5, 3]

In [42]:
intRDD.takeSample(True,10,123)#参数True表示放回样本采样，10表示采样个数，123表示随机种子

[2, 5, 5, 5, 2, 1, 1, 2, 2, 1]

In [43]:
intRDD.takeSample(False,10,123)

[5, 1, 5, 2, 3]

### RDD统计运算

In [44]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [45]:
intRDD.stats()

(count: 5, mean: 3.2, stdev: 1.6, max: 5.0, min: 1.0)

In [46]:
intRDD.stdev()#标准差，

1.6

In [47]:
intRDD.sum()

16

## RDD的key-value基本转换 

In [48]:
kvRDD=sc.parallelize([(1,2),(3,4),(5,6),(5,7)])#创建RDD key-value实例

In [49]:
kvRDD.collect()

[(1, 2), (3, 4), (5, 6), (5, 7)]

In [50]:
kvRDD.keys().collect()

[1, 3, 5, 5]

In [51]:
kvRDD.values().collect()

[2, 4, 6, 7]

In [52]:
kvRDD.filter(lambda kv:kv[0]<4).collect()#筛选出key值小于4的元素

[(1, 2), (3, 4)]

In [53]:
kvRDD.filter(lambda kv:kv[1]<4).collect()#筛选出value值小于4的元素

[(1, 2)]

### mapValues针对RDD内的每一组(key,value)的value进行运算，并且产生新的RDD

In [54]:
kvRDD.mapValues(lambda x:x*2).collect()

[(1, 4), (3, 8), (5, 12), (5, 14)]

In [55]:
kvRDD.sortByKey(ascending=False).collect()

[(5, 6), (5, 7), (3, 4), (1, 2)]

## 多个key-value的转换运算

In [56]:
kvRDD1=sc.parallelize([(3,4),(3,6),(5,6),(7,9)])
kvRDD2=sc.parallelize([(3,8)])

In [57]:
kvRDD1.collect()

[(3, 4), (3, 6), (5, 6), (7, 9)]

In [58]:
kvRDD2.collect()

[(3, 8)]

In [59]:
kvRDD1.join(kvRDD2).collect()

[(3, (4, 8)), (3, (6, 8))]

In [60]:
kvRDD1.leftOuterJoin(kvRDD2).collect()

[(3, (4, 8)), (3, (6, 8)), (5, (6, None)), (7, (9, None))]

In [61]:
kvRDD1.rightOuterJoin(kvRDD2).collect()

[(3, (4, 8)), (3, (6, 8))]

In [62]:
kvRDD1.subtractByKey(kvRDD2).collect()

[(5, 6), (7, 9)]

In [64]:
kvRDD1.countByKey()

defaultdict(int, {3: 2, 5: 1, 7: 1})

In [65]:
kvRDD1.collectAsMap()

{3: 6, 5: 6, 7: 9}

### Key-Value lookup运算 输入key查找value值

In [66]:
kvRDD1.collect()

[(3, 4), (3, 6), (5, 6), (7, 9)]

In [67]:
kvRDD1.lookup(3)

[4, 6]

In [68]:
kvRDD1.lookup(7)

[9]

## broadcast广播变量 

In [69]:
kvFruit=sc.parallelize([(1,'apple'),(2,'orange'),(3,'banana'),(4,'grap')])

In [70]:
fruitmap=kvFruit.collectAsMap()
fruitmap

{1: 'apple', 2: 'orange', 3: 'banana', 4: 'grap'}

In [71]:
fruitIds=sc.parallelize([2,4,1,3])
print('水果编号：'+str(fruitIds.collect()))

水果编号：[2, 4, 1, 3]


In [72]:
fruitNames=fruitIds.map(lambda x:fruitmap[x]).collect()

In [73]:
fruitNames

['orange', 'grap', 'apple', 'banana']

In [80]:
fruitmap

{1: 'apple', 2: 'orange', 3: 'banana', 4: 'grap'}

In [81]:
bcfruitmap=sc.broadcast(fruitmap)#转换成广播变量

In [83]:
bcfruitmap.value

{1: 'apple', 2: 'orange', 3: 'banana', 4: 'grap'}

In [89]:
fruitIds.collect()

[2, 4, 1, 3]

In [90]:
fruitNames1=fruitIds.map(lambda x:bcfruitmap.value[x]).collect()
fruitNames1

['orange', 'grap', 'apple', 'banana']

### accumulator累加器

In [92]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [104]:
total=sc.accumulator(0.0)#创建累加器，并设置初始值

In [105]:
intRDD.foreach(lambda x:total.add(x))#执行该动作，foreach遍历intRDD中的每个元素进行累加并将结果存储在累加器中，
                                                             #在foreach循环中无法使用.vlue方法获取结果，只能在foreach外获取结果

In [106]:
total.value

16.0

## RDD Persistence持久化 

持久化等级说明：
https://www.cnblogs.com/mahailuo/p/9564767.html

In [107]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [108]:
intRDD.is_cached#查看是否已缓存到内存中

False

In [109]:
intRDD.persist()

ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:175

In [110]:
intRDD.is_cached

True

In [111]:
intRDD.unpersist()

ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:175

In [112]:
intRDD.is_cached

False

In [120]:
from pyspark.storagelevel import StorageLevel
intRDD.persist(storageLevel=StorageLevel.MEMORY_AND_DISK)

ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:175

In [121]:
intRDD.is_cached

True

In [122]:
intRDD.unpersist()

ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:175